## scikit-learn 02 p111

### GridSearchCV: 교차 검증 + 하이퍼 파라미터 튜닝
하이퍼 파라미터: Machine learning algorithm을 구성하는 주요 구성 요소, 이 값을 조정해 알고리즘의 예측 성능 개선

GridSearchCV: Classifier나 Regressor와 같은 알고리즘에 사용되는 파이퍼 파리마터를 순차적으로 입력하면서 편리하게 최적의 파라미터를 도출


In [1]:
grid_parameters = {'max_depth': [1, 2, 3], 'min_samples_split': [2,3]}


In [10]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split


# 데이터 로딩, 학습 데이터와 테스트 데이터 분리
iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.2, random_state=121)

dtree = DecisionTreeClassifier()

# parameter를 dictionary 형태로 설정
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

In [15]:
import pandas as pd

# param_grid의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 테스트 수행 설정
### refit=True가 default. True이면 가장 좋은 파라미터 설정으로 재학습
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

# 학습 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습/평가
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과를 추출해 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [16]:
print('GridSearchCF Optimal parameter:', grid_dtree.best_params_)
print('GridSearchCV best accuracy: {0:.4f}'.format(grid_dtree.best_score_))

GridSearchCF Optimal parameter: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV best accuracy: 0.9750


In [18]:
from sklearn.metrics import accuracy_score

# GridSearchCV의 REFIT으로 이미 학습된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 학습이 됬으므로 별도 학습이 필요 없음
pred = estimator.predict(X_test)
print('test data set accuracy: {0:.4f}'.format(accuracy_score(y_test, pred)))

# 학습 데이터를 GridSearchCV를 이용해 최적 하이퍼 파라미터 튜닝을 수행
# 별도의 테스트 세트에서 이를 평가하는 것이 일반적인 머신러닝 모델 적용 방법

test data set accuracy: 0.9667


## Data Preprocessing p116

Scikit-learn machine learning algorithm은 문자열 값을 입력값으로 허용하지 않는다. 모든 문자열 값은 인코딩돼서 숫자 형으로 변환

## Data Encoding
lable encoding: 카테고리 피처를 코드형 숫자값으로 변환하는 것

one hot encoding: 피처 값의 유형에 따라 새로운 피처를 추가해 고유 값에 해당하는 칼럼에만 1을 표시하고 나머지 칼럼에는 0을 표시하는 방식

In [19]:
from sklearn.preprocessing import LabelEncoder

items = ['TV', 'Refrigerator', 'Microwave', 'Computer', 'Fan', 'Fan', 'Mixer', 'Mixer']

# LabelEncoder를 객체로 생성한 후, fit()과 transform()으로 label encodeing 수행
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('Encoding values:', labels)

Encoding values: [5 4 2 0 1 1 3 3]


In [20]:
print('Encoding class:', encoder.classes_)

Encoding class: ['Computer' 'Fan' 'Microwave' 'Mixer' 'Refrigerator' 'TV']


In [22]:
print('Decoding original:',encoder.inverse_transform([5, 4, 2, 0, 1, 1, 3, 3]))

Decoding original: ['TV' 'Refrigerator' 'Microwave' 'Computer' 'Fan' 'Fan' 'Mixer' 'Mixer']


In [23]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

items = ['TV', 'Refrigerator', 'Microwave', 'Computer', 'Fan', 'Fan', 'Mixer', 'Mixer']

# Transform 2d array
items = np.array(items).reshape(-1, 1)

# OneHotEncoding
oh_encoder = OneHotEncoder()
oh_encoder.fit(items)
oh_labels = oh_encoder.transform(items)

# OneHotEncoder로 변환한 결과는 희소행렬이므로 toarray()를 이용해 밀집 행렬로 변환
print('OneHotEncoding data')
print(oh_labels.toarray())
print('OneHotEncoding data dimension')
print(oh_labels.shape)

OneHotEncoding data
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]
OneHotEncoding data dimension
(8, 6)


In [24]:
import pandas as pd

df = pd.DataFrame({'item':['TV', 'Refrigerator', 'Microwave', 'Computer', 'Fan', 'Fan', 'Mixer', 'Mixer']})
pd.get_dummies(df)

,item_Computer,item_Fan,item_Microwave,item_Mixer,item_Refrigerator,item_TV
0,0,0,0,0,0,1
1,0,0,0,0,1,0
2,0,0,1,0,0,0
3,1,0,0,0,0,0
4,0,1,0,0,0,0
5,0,1,0,0,0,0
6,0,0,0,1,0,0
7,0,0,0,1,0,0


## Feagure scaling and Standardization p122

### StandardScaler

In [27]:
from sklearn.datasets import load_iris
import pandas as pd
# data set load and transform it to DataFrame
iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)

print('feature average')
print(iris_df.mean())
print('\nfeature variance (분산)')
print(iris_df.var())

feature average
sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64

feature variance (분산)
sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


In [28]:
from sklearn.preprocessing import StandardScaler

# StandardScaler object 
scaler = StandardScaler()
# StandardScaler로 데이터 세트 변환. fit(), transform() call
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transform()시 스케일 변환된 데이터 세트가 Numpy ndarray로 반환돼 이를 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)
print('feature average')
print(iris_df_scaled.mean())
print('\nfeature variance')
print(iris_df_scaled.var())

feature average
sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64

feature variance
sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


## MinMaxScaler

데이터값을 0과 1 사이의 범위 값으로 변환

In [29]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler object creation
scaler = MinMaxScaler()
# MinMaxScaler로 데이터 세트 변환. fit() and transform()
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transform() 시 스케일 변환된 데이터 세트가 Numpy ndarray로 변환되어 이를 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)
print('feature minimum')
print(iris_df_scaled.min())
print('\nfeature maximum')
print(iris_df_scaled.max())

feature minimum
sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64

feature maximum
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


In [30]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 학습 데이터는 0부터 10까지, 테스트 데이터는 0부터 5까지 값을 가지는 데이터 세트로 생성
# Scaler class의 fit(), transform()은 2차원 이상 데이터만 가능하므로 reshape(-1, 1)로 차원 변경
train_array = np.arange(0, 11).reshape(-1, 1)
test_array = np.arange(0, 6).reshape(-1, 1)

In [31]:
#MinMaxScaler 객체에 별도의 feature_range parameter값을 지정하지 않으면 0~1값으로 변환
scaler = MinMaxScaler()

# fit()하게 되면 train_array 데이터의 최소값이 0, 최대값이 10으로 설정
scaler.fit(train_array)

# 1/10 scale로 train_array 데이터 변환함. 원본 10 -> 1로 변환됨
train_scaled = scaler.transform(train_array)

print('원본 train_array data: ', np.round(train_array.reshape(-1), 2))
print('scaled train_array: ', np.round(train_scaled.reshape(-1), 2))

원본 train_array data:  [ 0  1  2  3  4  5  6  7  8  9 10]
scaled train_array:  [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [32]:
# MinMaxScaler에 test_array를 fit()하게 되면 원본 데이터의 최소값이 0, 최대값이 5로 설정됨
scaler.fit(test_array)

# 1/5 scale 로 test_array 데이터 반환함. 원본 5->1 로 변환
test_scaled = scaler.transform(test_array)

# test_array의 scale 변환 출력
print('original test_array data: ', np.round(test_array.reshape(-1), 2))
print('Scaled test_array data: ', np.round(test_scaled.reshape(-1),2))

original test_array data:  [0 1 2 3 4 5]
Scaled test_array data:  [0.  0.2 0.4 0.6 0.8 1. ]


In [33]:
scaler = MinMaxScaler()
scaler.fit(train_array)
train_scaled = scaler.transform(train_array)
print('Original train_array data: ', np.round(train_array.reshape(-1), 2))
print('Scaled train_array data: ', np.round(train_scaled.reshape(-1), 2))

# test_array에 scale 변환을 할 때에는 반드시 fit()을 호출하지 않고 transform()만으로 변환해야 함
test_scaled = scaler.transform(test_array)
print('\n Original test_array data: ', np.round(test_array.reshape(-1), 2))
print('Scaled test_array data: ', np.round(test_scaled.reshape(-1), 2))

Original train_array data:  [ 0  1  2  3  4  5  6  7  8  9 10]
Scaled train_array data:  [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]

 Original test_array data:  [0 1 2 3 4 5]
Scaled test_array data:  [0.  0.1 0.2 0.3 0.4 0.5]
